# Parametrized Witness

An _entanglement witness_ is a functional which distinguishes a specific entangled state from separable ones.

In this notebook we will use a parametrized witness of the form
$$
    W(\alpha, \beta, \gamma, \delta, \epsilon) =
    \alpha I - \left| \psi(\beta, \gamma, \delta, \epsilon) \right\rangle \left\langle \psi(\beta, \gamma, \delta, \epsilon) \right|
$$
as seen in "Machine-Learning-Derived Entanglement Witnesses" by _Greenwood et al._ (2023).

We will then proceed to optimize the parameters $\alpha, \beta, \gamma, \delta, \epsilon$ to find the witness that best distinguishes entangled from separate states

In [1]:
# imports

import pandas as pd
from scipy.optimize import minimize
import numpy as np

In [6]:
# Constants
N_DATA = 1000

Bell_state_0 = 1/np.sqrt(2)*(np.array([1,0,0,0])+np.array([0,0,0,1]))
Bell_state_1 = 1/np.sqrt(2)*(np.array([1,0,0,0])-np.array([0,0,0,1]))
Bell_state_2 = 1/np.sqrt(2)*(np.array([0,1,0,0])+np.array([0,0,1,0]))
Bell_state_3 = 1/np.sqrt(2)*(np.array([0,1,0,0])-np.array([0,0,1,0]))

Bell_0 = np.outer(Bell_state_0, Bell_state_0)
Bell_1 = np.outer(Bell_state_1, Bell_state_1)
Bell_2 = np.outer(Bell_state_2, Bell_state_2)
Bell_3 = np.outer(Bell_state_3, Bell_state_3)

Bell_basis = np.stack([Bell_0, Bell_1, Bell_2, Bell_3])
Bell_basis_state = np.stack([Bell_state_0, Bell_state_1, Bell_state_2, Bell_state_3])

## Dataset

The dataset consists of operators. Since the operator are applied to the state |00>, the state generated by the operator is given by the first column.
Witness works with density matrices, so we will convert the state into a density matrix.


In [8]:
dataset_U = pd.read_csv("../datasets/ds_haar_op.csv")
dataset_U = dataset_U[:N_DATA]

# Separate features (X) and labels (y)

# Drop the 17th column
X = dataset_U.drop(columns=dataset_U.columns[16])

y = dataset_U.iloc[:, 16]  # Assuming the label is in the 17th column (index 16)
y = y.to_numpy(dtype=int) # convert the labels in torch tensor

dataset_psi_out = X.iloc[:, :4] # select only the first row of the unitary matrix, because it represent the output state after apply U to |00>
dataset_psi_out_np = dataset_psi_out.to_numpy(dtype=np.csingle) # convert the dataset in numpy

# compute the density matrices dataset as a torch tensor 
dens_matrices = np.array([(np.outer(dataset_psi_out_np[i], np.conj(dataset_psi_out_np[i]))) for i in range(N_DATA)])

## Optimization

We will use the `minimize` function from `scipy.optimize` to find the parameters that minimize the error function.

In [16]:
def error(coefficent, X, Y):

    N = X.shape[0]
    err = 0

    for i in range(N):
        psi = coefficent[0]*Bell_basis_state[0] + coefficent[1]*Bell_basis_state[1] + coefficent[2]*Bell_basis_state[2] + coefficent[3]*Bell_basis_state[3]
        W = np.eye(4) - np.outer(np.conj(psi).T, psi)
        pred = np.real(np.trace(W@X[i])) >= 0

        err += np.abs(Y[i] - pred)
    # print(f"Error: {err/N}")
    return err/N

def mse(coefficent, X, Y):

    N = X.shape[0]
    err = 0

    for i in range(N):
        psi = coefficent[0]*Bell_basis_state[0] + coefficent[1]*Bell_basis_state[1] + coefficent[2]*Bell_basis_state[2] + coefficent[3]*Bell_basis_state[3]
        W = np.eye(4) - np.outer(np.conj(psi).T, psi)
        pred = np.real(np.trace(W@X[i])) >= 0

        err += (Y[i] - pred)**2
    # print(f"Error: {err/N}")
    return err/N

options = {'maxiter': 100, 'disp': True}

In [17]:
# restart various times the optimization to avoid local minima
accuracies = []
for _ in range(10):
    # generate an array of size 4 with random complex numbers with values between -1 and 1
    coefficent = np.random.uniform(-1, 1, 4) + 1j*np.random.uniform(-1, 1, 4)
    
    # minimize the error function
    result = minimize(error, coefficent, args=(dens_matrices, y), method='Powell', options=options)
    
    # get score
    score = 1 - result.fun
    
    # append the score to the list
    accuracies.append(score)

print(f"\nBest accuracy: {max(accuracies)}")

Optimization terminated successfully.
         Current function value: 0.439000
         Iterations: 4
         Function evaluations: 400
Optimization terminated successfully.
         Current function value: 0.459000
         Iterations: 4
         Function evaluations: 406
Optimization terminated successfully.
         Current function value: 0.467000
         Iterations: 5
         Function evaluations: 536
Optimization terminated successfully.
         Current function value: 0.446000
         Iterations: 3
         Function evaluations: 304
Optimization terminated successfully.
         Current function value: 0.430000
         Iterations: 4
         Function evaluations: 287
Optimization terminated successfully.
         Current function value: 0.445000
         Iterations: 2
         Function evaluations: 229
Optimization terminated successfully.
         Current function value: 0.444000
         Iterations: 3
         Function evaluations: 404
Optimization terminated successful